# Advanced Lane Lines - Perspective transformation

In this notebook the following points from the Project 4 Advanced Lane Lines are implemented:
* Generating output video.
 

---
## Step 0: Import required libraries 

In [6]:
import cv2
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib inline

from moviepy.editor import VideoFileClip
from IPython.display import HTML

from camera_calibration import undistortImage, calibrateCamera
from color_gradient import imageTransformation
from perspective_transformation import getPerspectiveTransform, warpImage2birdsEyeView, pipelineImageTransformation
from lane_detection import Line, window, getLaneIndices, getLanePixelPositions, detectLaneLines, projectLaneLinesRoad, calculateAndWriteCurvatureRadius, caculateAndWriteLaneOffset


## Step 2:  Track lane lines function

In [7]:
def trackLaneLines(birdsEyeView, line_left, line_right):
    nonzero = birdsEyeView.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    out_img = (np.dstack((birdsEyeView, birdsEyeView, birdsEyeView))*255).astype(np.uint8)
            
    margin = 100  # dict_config_params['x_margin']
        
    left_fit = line_left.getFit(useAverageFit=True)
    right_fit = line_right.getFit(useAverageFit=True)
    
    x_left = left_fit[0] * (nonzeroy**2) + left_fit[1] * nonzeroy + left_fit[2]
    x_right = right_fit[0] * (nonzeroy**2) + right_fit[1] * nonzeroy + right_fit[2]
        
    w_left = window(x_left - margin, x_left + margin, nonzeroy.min(), nonzeroy.max()) 
    w_right = window(x_right - margin, x_right + margin, nonzeroy.min(), nonzeroy.max()) 
    
    left_lane_inds, right_lane_inds = getLaneIndices(nonzero, w_left, w_right)
    
    pixel_pos_x, pixel_pos_y = getLanePixelPositions(nonzero, left_lane_inds)
    line_left.updateLineFit(pixel_pos_x, pixel_pos_y)
    
    pixel_pos_x, pixel_pos_y = getLanePixelPositions(nonzero, right_lane_inds)
    line_right.updateLineFit(pixel_pos_x, pixel_pos_y)
 
    ploty, left_fitx = line_left.getXY(birdsEyeView.shape[0], useAverageFit=True)
    ploty, right_fitx = line_right.getXY(birdsEyeView.shape[0], useAverageFit=True)
    
    # Color lane-pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    ## Draw search windows for the left and right lane lines
    window_img = np.zeros_like(out_img)
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_window_left_line = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_window_right_line = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_window_pts = np.hstack((left_window_left_line, left_window_right_line))
    
    right_window_left_line = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_window_right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_window_pts = np.hstack((right_window_left_line, right_window_right_line))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_window_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_window_pts]), (0,255, 0))
    out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    return out_img

## Step 3:  Initialize variables containing camera calibration and prespective transformation parameters 

In [8]:
def initialize():
    # Calibrate camera
    mtx, dist = calibrateCamera(useCalibrationCache=True)
    
    # Perspective transform
    image = cv2.cvtColor(cv2.imread('./test_images/test2.jpg'), cv2.COLOR_BGR2RGB)
    imageUndistored = undistortImage(image, mtx, dist, plotImages=False)
    imageSize = (imageUndistored.shape[1], imageUndistored.shape[0])
    src, dst, perspective_M, perspective_M_inv = getPerspectiveTransform(imageSize)
    
    return mtx, dist, perspective_M, perspective_M_inv

## Step 4:  Helper function processing video images/frames

In [9]:
# Process the `inputVideo` image by image/frame by frame to find the lane lines, draw curvarute and vehicle position information 

def processVideoImage(image):
    global configParams
    global numberImagesProcessed    
    global mtx, dist, perspective_M, perspective_M_inv
    global leftFit, rightFit, leftFitx, rightFitx 
    
    #undistoredImage = undistortImage(image, mtx, dist, plotImages=False)
    #imageTransformed = imageTransformation(undistoredImage)
    
    #imageSize = (imageTransformed.shape[1], imageTransformed.shape[0])
    #birdsEyeView = warpImage2birdsEyeView(imageTransformed.astype(np.uint8), 
                                   #imageSize, perspective_M).astype(bool)
    
    undistoredImage = undistortImage(image, mtx, dist, plotImages=False)
    birdsEyeView = pipelineImageTransformation(image).astype(bool)

    out_img = None
    ploty = None
    if numberImagesProcessed==0:        
        outputImage = detectLaneLines(birdsEyeView, lineLeft, lineRight, plotImage=False)         
    else:        
        outputImage = trackLaneLines(birdsEyeView, lineLeft, lineRight)
    
    numberImagesProcessed += 1
        
    imageLinesRoad = projectLaneLinesRoad(undistoredImage, outputImage, lineLeft, lineRight, perspective_M_inv)
    calculateAndWriteCurvatureRadius(imageLinesRoad, configParams, lineLeft, lineRight)
    caculateAndWriteLaneOffset(imageLinesRoad, configParams, lineLeft, lineRight)
        
    return imageLinesRoad


## Step 5:  Process and generate output video

In [14]:
# Config parameters
configParams = {'x_margin': 100,
                'y_meter_per_pixel': 30.0/720,
                'x_meter_per_pixel': 3.7/700,
                }

leftFit, rightFit, leftFitx, rightFitx = None, None, None, None
mtx, dist, perspective_M, perspective_M_inv = None, None, None, None
lineLeft = None
lineRight = None

# Reset global
numberImagesProcessed = 0

if __name__ == '__main__':

    mtx, dist, perspective_M, perspective_M_inv = initialize()
    
    lineLeft = Line()
    lineRight = Line()

    #clip1 = VideoFileClip("./videos/project_video.mp4").subclip(0,2)
    #clip = clip1.fl_image(processVideoImage)
    #clip.write_videofile("output_project_video_short.mp4", audio=False)
    
    clip1 = VideoFileClip("./videos/project_video.mp4")
    clip = clip1.fl_image(processVideoImage)
    clip.write_videofile("output_project_video.mp4", audio=False)
    
    #clip1 = VideoFileClip("./videos/challenge_video.mp4")
    #clip = clip1.fl_image(processVideoImage)
    #clip.write_videofile("output_challenge_video.mp4", audio=False)
    
    # Reset global
    numberImagesProcessed = 0 


Using already available cached calibration results.

[MoviePy] >>>> Building video output_project_video_short.mp4
[MoviePy] Writing video output_project_video_short.mp4




  0%|          | 0/51 [00:00<?, ?it/s]

  2%|▏         | 1/51 [00:00<00:17,  2.88it/s]

  4%|▍         | 2/51 [00:01<00:23,  2.08it/s]

  6%|▌         | 3/51 [00:01<00:22,  2.17it/s]

  8%|▊         | 4/51 [00:01<00:19,  2.40it/s]

 10%|▉         | 5/51 [00:02<00:17,  2.61it/s]

 12%|█▏        | 6/51 [00:02<00:16,  2.72it/s]

 14%|█▎        | 7/51 [00:02<00:15,  2.75it/s]

 16%|█▌        | 8/51 [00:03<00:15,  2.79it/s]

 18%|█▊        | 9/51 [00:03<00:15,  2.74it/s]

 20%|█▉        | 10/51 [00:03<00:14,  2.78it/s]

 22%|██▏       | 11/51 [00:04<00:14,  2.82it/s]

 24%|██▎       | 12/51 [00:04<00:13,  2.89it/s]

 25%|██▌       | 13/51 [00:05<00:14,  2.66it/s]

 27%|██▋       | 14/51 [00:05<00:17,  2.15it/s]

 29%|██▉       | 15/51 [00:06<00:17,  2.03it/s]

 31%|███▏      | 16/51 [00:06<00:16,  2.17it/s]

 33%|███▎      | 17/51 [00:07<00:14,  2.33it/s]

 35%|███▌      | 18/51 [00:07<00:13,  2.44it/s]

 37%|███▋      | 19/51 [00:07<00:12,  2.55it/s]

 39%|███▉      | 20/51 [00:08<00:12,

[MoviePy] Done.
[MoviePy] >>>> Video ready: output_project_video_short.mp4 

